In [6]:
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 39.0 MB/s eta 0:00:00m eta 0:00:010:01:01


In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),       # Resize images to 224x224
    transforms.ToTensor(),               # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

In [7]:

# Load Flowers102 dataset
train_data = datasets.Flowers102(
    root="./data",  # Directory to store data
    split="train",  # Options: 'train', 'val', 'test'
    transform=transform,
    download=True   # Download if not already downloaded
)

val_data = datasets.Flowers102(
    root="./data",
    split="val",
    transform=transform,
    download=True
)

test_data = datasets.Flowers102(
    root="./data",
    split="test",
    transform=transform,
    download=True
)

In [8]:
# Data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Check dataset size
print(f"Train size: {len(train_data)}")
print(f"Validation size: {len(val_data)}")
print(f"Test size: {len(test_data)}")

Train size: 1020
Validation size: 1020
Test size: 6149


In [16]:
import torch.nn as nn
from torchvision import models
import torch.optim as optim

In [18]:
# Load pre-trained model
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)
model = models.resnet50(pretrained=True)

# Modify the classifier
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 102)  # 102 classes
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

# Save the model
torch.save(model.state_dict(), "flower_model.pth")


mps
Epoch 1/50, Loss: 4.624006867408752
Epoch 2/50, Loss: 2.665600948035717
Epoch 3/50, Loss: 1.562186848372221
Epoch 4/50, Loss: 0.9698489811271429
Epoch 5/50, Loss: 0.6590888118371367
Epoch 6/50, Loss: 0.44780580420047045
Epoch 7/50, Loss: 0.3612577994354069
Epoch 8/50, Loss: 0.27846274012699723
Epoch 9/50, Loss: 0.20784446084871888
Epoch 10/50, Loss: 0.1786797815002501
Epoch 11/50, Loss: 0.15118100075051188
Epoch 12/50, Loss: 0.12677217344753444
Epoch 13/50, Loss: 0.10896958340890706
Epoch 14/50, Loss: 0.08655079174786806
Epoch 15/50, Loss: 0.07926300552207977
Epoch 16/50, Loss: 0.06937166478019208
Epoch 17/50, Loss: 0.06202934333123267
Epoch 18/50, Loss: 0.059449083637446165
Epoch 19/50, Loss: 0.054006511345505714
Epoch 20/50, Loss: 0.05080961843486875
Epoch 21/50, Loss: 0.048546064645051956
Epoch 22/50, Loss: 0.04309162142453715
Epoch 23/50, Loss: 0.038092856004368514
Epoch 24/50, Loss: 0.036885363631881773
Epoch 25/50, Loss: 0.031073131773155183
Epoch 26/50, Loss: 0.0333952606888